In [32]:
import numpy as np

In [33]:
class Conv:
    def __init__(self, seed = None):
        if seed is not None:
            self.seed = seed
        
    def Conv1D(self, X, kernel_size: int):
        self.X = X
        self.kernel_size = kernel_size
        self.kernel = np.random.random_sample(size = (kernel_size))
        X_size = self.X.size
        output_width = X_size - kernel_size + 1
        Y = np.zeros(shape = (output_width))
        for i in range(Y.size):
            Y[i] = np.sum(self.X[i:i + kernel_size] * self.kernel)
        return Y
  
    def Conv1D_pad(self, X, kernel_size, padding_size):
        self.padding_size = padding_size
        self.X = np.pad(X, pad_width = padding_size)
        self.kernel_size = kernel_size
        self.kernel = np.random.random_sample(size = (kernel_size))
        X_size = self.X.size
        output_width = X_size - kernel_size + 1
        Y = np.zeros(shape = (output_width))
        for i in range(Y.size):
            Y[i] = np.sum(self.X[i:i + kernel_size] * self.kernel)
        return Y
     
    def Conv1D_pad_stride(self, X, kernel_size, padding_size, stride):
        self.padding_size = padding_size
        self.stride = stride
        self.X = np.pad(X, pad_width = padding_size)
        self.kernel_size = kernel_size
        self.kernel = np.random.random_sample(size = (kernel_size))
       
        X_size = self.X.size
        
        output_size = int(((X_size + (2 * padding_size)- kernel_size) / (stride)) + 1)
        Y = np.zeros(shape = (output_size))
      
        for i in range(Y.size):
            Y[i] = np.sum(self.X[i * self.stride:i * self.stride +kernel_size] * self.kernel)
        return Y
  
    def Conv1D_pad_stride_dilation(self, X, kernel_size, padding_size, stride, dilation_rate):
        self.padding_size = padding_size
        self.stride = stride
        self.dilation_rate = dilation_rate
        self.X = np.pad(X, pad_width=padding_size)

        kernel_mask = np.random.random_sample(size=(kernel_size))
        self.kernel = self._dilate_1D(kernel_mask, dilation_rate=self.dilation_rate)
        
        output_size = int(((self.X.size - self.kernel.size) / self.stride) + 1)
        Y = np.zeros(shape=(output_size))

        for i in range(Y.size):
            current_slice = self.X[i * self.stride : (i * self.stride + self.kernel.size)]

            Y[i] = np.sum(current_slice * self.kernel) 

        return Y  
  
    def Conv1D_pad_stride_dilation_channels(self, X, kernel_size, output_channels, padding_size, stride, dilation_rate):
        
        '''
        X is multiple channels, shape: (Input Channels, Feats per Channel)
        Kernel is multiple channels, shape: (Output Channels, Input Channels, Params per Channel)
        ''' 
        
        self.padding_size = padding_size
        self.stride = stride
        self.dilation_rate = dilation_rate
        self.X = np.pad(X, pad_width=padding_size)

        
        kernel_mask = np.random.random_sample(size=(output_channels, X.shape[0], kernel_size))
        self.kernel = self._dilate_1D_multiple_channels(kernel_mask, dilation_rate)
        
        output_size = int(((self.X.shape[1] - self.kernel.shape[2]) / self.stride) + 1)
     
        Y = np.zeros(shape=(output_channels, output_size))
        

        for out_ch in range(Y.shape[0]): # solves multiple output channels
            for in_ch in range(X.shape[0]):
                for i in range(Y.shape[1]): # solves multiple output channels
                    
                    slice_idx = i*self.stride 
                
                    current_slice = self.X[in_ch, slice_idx:(slice_idx + self.kernel.shape[2])]
                    conv_out = []
                    
                    if current_slice.size != self.kernel.shape[2]:
                        break 
                    
                    for in_ch in range(self.kernel.shape[1]): 
                        conv_ch = current_slice * self.kernel[out_ch, in_ch]
                        conv_out.append(conv_ch)
                    
                    Y[out_ch, i] = np.sum(conv_out)
        

        print(f"Input:\n{X}\n")
        print(f"Kernel:\n{self.kernel}\n")
        print(f"Stride:{stride}\n")
        print(f"Output Channels: {output_channels}\n")
        print(f"Output:\n{Y}")
        return Y

     
        
    def _dilate_1D(self, kernel, dilation_rate = 2):
        dilation_rate -= 1
        i = 0
        
        if len(kernel) == 1:
            return kernel
        
        while i < len(kernel):
            if kernel[i] != 0:
                kernel = np.concatenate((kernel[:i+1], [0 for _ in range(dilation_rate)], kernel[i+1:])) # works instead of X[i:], as we want to exclude the ith element for the end. we already including that in the former part of the X array.
                i += dilation_rate
            i += 1
            if i == (len(kernel) - 1):
                return kernel
   
    def _dilate_1D_multiple_channels(self, kernel, dilation_rate = 2):
               
        if kernel.shape[2] == 1:
            return kernel
      
        out_kernel = np.zeros(shape = ( kernel.shape[0], kernel.shape[1], (kernel.shape[2] * dilation_rate - (dilation_rate - 1)))) 
     
        dilation_rate -= 1
        
        for out_ch in range(kernel.shape[0]): 
            for in_ch in range(kernel.shape[1]):
                i = 0
                dilated_row = kernel[out_ch, in_ch, :]
                while i < len(dilated_row):
                    if dilated_row[i] != 0:
                        dilated_row = np.concatenate((dilated_row[:i+1], [0 for _ in range (dilation_rate)], dilated_row[i+1:]))
                        i += dilation_rate
                    i += 1
                    if i == (len(dilated_row) - 1):
                        out_kernel[out_ch, in_ch, :] = dilated_row
                        break
        return out_kernel
    
    @property
    def seed(self):
        return self._seed
    
    @seed.setter
    def seed(self, seed):
        np.random.seed(seed = seed)
        self._seed = seed
    
    @property
    def kernel_size(self):
        return self._kernel_size     
   
    @kernel_size.setter
    def kernel_size(self, kernel_size):
        assert self.X.size >= kernel_size, ValueError('Kernel cannot be greater than input_vector!')
        assert isinstance(kernel_size, int), ValueError('kernel_size must be int for 1D Conv')
        self._kernel_size = kernel_size
       

In [34]:
seed = 1
ops = Conv(seed = seed)

In [35]:
x = np.array([2, 3, 2, 1, 2])
print(f"Seed: {seed}") 
print(f"Input: {x}")

Seed: 1
Input: [2 3 2 1 2]


In [36]:
Y = ops.Conv1D(X = x, kernel_size=2)
print(f"ops.Conv1D(X = x, kernel_size=2)\n{Y}")

ops.Conv1D(X = x, kernel_size=2)
[2.99501749 2.691715   1.5543685  1.85767099]


In [37]:
Y = ops.Conv1D_pad(X = x, kernel_size = 2, padding_size=1)
print(f"ops.Conv1D_pad(X = x, kernel_size = 2, padding_size=1):\n{Y}")

ops.Conv1D_pad(X = x, kernel_size = 2, padding_size=1):
[6.04665145e-01 9.07226468e-01 6.05008270e-01 3.02561322e-01
 6.04779520e-01 2.28749635e-04]


In [38]:
Y = ops.Conv1D_pad_stride(X = x, kernel_size = 2, padding_size=0, stride = 2)
print(f"ops.Conv1D_pad_stride(X = x, kernel_size = 2, padding_size=0, stride = 2):\n{Y}")

ops.Conv1D_pad_stride(X = x, kernel_size = 2, padding_size=0, stride = 2):
[0.57052757 0.38585038]


In [39]:
kernel = np.array([1, 2, 3, 4])
dilated_kernel =  ops._dilate_1D(kernel = kernel, dilation_rate = 2)
print(f"ops._dilate_1D(X = x, dilation_rate=2):\n{dilated_kernel}\n")

ops._dilate_1D(X = x, dilation_rate=2):
[1 0 2 0 3 0 4]



In [40]:
x = np.array([2, 3, 4, 5, 1, 2, 3, 4, 1, 2])
print(f"ops.Conv1D_pad_stride_dilation(x, kernel_size=2, padding_size = 2, stride = 2, dilation_rate = 2):")
Y = ops.Conv1D_pad_stride_dilation(x, kernel_size=2, padding_size = 2, stride = 2, dilation_rate = 2) 
print(f"{Y}\n")

ops.Conv1D_pad_stride_dilation(x, kernel_size=2, padding_size = 2, stride = 2, dilation_rate = 2):
[0.69112145 1.75476333 1.09060157 1.22294239 0.90434136 0.18626021]



In [41]:
x = np.array([[1, 2, 3, 4], [5, 6, 7, 8]])
Y = ops.Conv1D_pad_stride_dilation_channels(X = x, kernel_size = 4, output_channels = 3 ,padding_size = 0, stride = 1, dilation_rate=1)

Input:
[[1 2 3 4]
 [5 6 7 8]]

Kernel:
[[[0.39676747 0.53881673 0.41919451 0.6852195 ]
  [0.20445225 0.87811744 0.02738759 0.67046751]]

 [[0.4173048  0.55868983 0.14038694 0.19810149]
  [0.80074457 0.96826158 0.31342418 0.69232262]]

 [[0.87638915 0.89460666 0.08504421 0.03905478]
  [0.16983042 0.8781425  0.09834683 0.42110763]]]

Stride:1

Output Channels: 3

Output:
[[25.47927448]
 [25.55202594]
 [20.83262944]]
